In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import transformers
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
gpus = tf.config.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print('GPU enable')
    except Exception as e:
        print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU enable


### Analog ```tf.keras.utils.image_dataset_from_directory```

In [3]:
path = './archive/'
train_path = f'{path}train/'
test_path = f'{path}test/'
valid_path = f'{path}valid/'

In [4]:
class_to_id = {}

def pd_from_path(directory: str) -> pd.DataFrame:
    global class_to_id
    data = []
    for path, _, files in os.walk(directory):
        for file in files:
            if file.endswith(('.jpg', '.png', '.jpeg', '.bmp', '.gif')):
                class_name = os.path.basename(path)
                if class_name not in class_to_id:
                    class_to_id[class_name] = len(class_to_id)
                data.append((f'{path}/{file}', class_to_id[class_name]))

    return pd.DataFrame(data, columns=['imagepath', 'label'])

In [ ]:
deleted = []
for file in pd_from_path(path).imagepath:
    try:
      img_bytes = tf.io.read_file(file)
      decoded_img = tf.io.decode_image(img_bytes)
    except Exception:
      deleted.append(file)
      os.remove(file)

In [6]:
def tf_load_image(image_path: str, crop_size: tuple[int, int] = (256, 256)):
    image = tf.io.read_file(image_path)
    image = tf.io.decode_image(image, expand_animations=False, channels=3)
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    image = tf.image.resize(image, size=[*crop_size])
    return image

In [7]:
def extract_and_zip(df: pd.DataFrame, crop_size: tuple[int, int] = (256, 256), batch_size: int = 32):
    image_dataset = tf.data.Dataset.from_tensor_slices(df.imagepath)
    label_dataset = tf.data.Dataset.from_tensor_slices(df.label)
    dataset = tf.data.Dataset.zip((image_dataset.map(lambda x: tf_load_image(x, crop_size)), label_dataset))
    return dataset.batch(batch_size)

In [8]:
def load_dataset_from_directory(
        directory: str = './',
        batch_size: int = 32,
        image_size: tuple[int, int] = (256, 256),
        shuffle: bool = True,
        seed: int = None,
        validation_split: float = None,
        subset: str = None
) -> tf.data.Dataset:
    
    df = pd_from_path(directory)
    
    if shuffle:
        df = df.sample(frac=1, random_state=seed)
    
    if validation_split:
        index = round(len(df) * validation_split)
        train_dataset = df[index:]
        valid_dataset = df[:index]

        if subset == 'training':
            df = train_dataset
        elif subset == 'validation':
            df = valid_dataset
        elif subset == 'both':
            return extract_and_zip(train_dataset), extract_and_zip(valid_dataset)
    
    
    return extract_and_zip(df, image_size, batch_size)
        

In [9]:
dataset = load_dataset_from_directory(path)

In [10]:
inputs = tf.keras.layers.Input((256, 256, 3), name='input')
x = tf.keras.layers.Conv2D(32, 3, padding='same')(inputs)
x = tf.keras.layers.PReLU()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(64)(x)

In [11]:
model = tf.keras.Model(inputs=inputs, outputs=x, name='ciaonima-1')
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics='accuracy', optimizer='adam')

In [12]:
model.fit(dataset, epochs=1)

2809/2809 [==============================] - 231s 81ms/step - loss: nan - accuracy: 0.0019


### Function

In [9]:
def load_dataset(path: str, batch_size: int, image_size: tuple[int, int], shuffle: bool, split: str) -> tuple[tf.data.Dataset, dict[int, str]]:
    '''Given a `path` to a csv index file loads one of the dataset splits. Paths in the index are assumed to be relative to the csv file. The file contains three columns: "filepaths", "labels" and "data set", path to the image, image label and dataset split respectively.

    Arguments:
        path: path to the csv index file
        batch_size: size of batches in the dataset
        image_size: size to resize the images to
        shuffle: whether to shuffle the index. If False original index order is preserved
        split: split to use. One of "train", "valid" or "test"

    Returns:
        The loaded dataset
        A dictionary mapping class indices to class names'''
    data = pd.read_csv(path)
    data.filepaths = [os.path.join(os.path.dirname(path), file).replace('\\', '/') for file in data.filepaths.tolist()]
    classes = pd.unique(data.labels)
    class_to_id = {name: i for i, name in enumerate(classes)}
    data.insert(3, 'labels_id', [class_to_id[name] for name in data.labels], True)

    if shuffle:
        data = data.sample(frac=1)
    
    if split:
        data = data[data['data set'] == split]

    image_dataset = tf.data.Dataset.from_tensor_slices(data.filepaths)
    label_dataset = tf.data.Dataset.from_tensor_slices(data.labels_id)
    
    dataset = tf.data.Dataset.zip((image_dataset.map(lambda x: tf_load_image(x, image_size)), label_dataset))

    return dataset.batch(batch_size), {i: name for i, name in enumerate(classes)}

In [10]:
dataset2, cindex = load_dataset('./archive/new_birds.csv', 32, (256, 256), False, None)

In [11]:
inputs2 = tf.keras.layers.Input((256, 256, 3), name='input')
x2 = tf.keras.layers.Conv2D(32, 3, padding='same')(inputs2)
x2 = tf.keras.layers.PReLU()(x2)
x2 = tf.keras.layers.Flatten()(x2)
x2 = tf.keras.layers.Dense(64)(x2)

In [12]:
model2 = tf.keras.Model(inputs=inputs2, outputs=x2, name='ciaonima-10')

In [13]:
model2.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics='accuracy', optimizer='adam')

In [14]:
model2.fit(dataset2, epochs=1)

2809/2809 [==============================] - 232s 82ms/step - loss: nan - accuracy: 0.0135


### Interface ```tf.keras.utils.Sequence```

In [21]:
class DatasetFromDirectory(tf.keras.utils.Sequence):

    def __init__(self,
                 directory: str = './',
                 batch_size: int = 32,
                 image_size: tuple[int, int] = (256, 256),
                 shuffle: bool = True,
                 seed: int = None,
                 validation_split: float = None,
                 subset: str = None,
                 augmentation: bool = True
                ):
        self.directory = directory
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle
        self.seed = seed
        self.validation_split = validation_split
        self.subset = subset
        self.augmentation = augmentation

        self.dataframe = self.pd_from_path(self.directory)
        
        if self.shuffle:
            self.dataframe = self.dataframe.sample(frac=1, random_state=self.seed)

        if self.validation_split:
            index = round(self.n * self.validation_split)
            train_dataset = self.dataframe[index:]
            valid_dataset = self.dataframe[:index]

            if self.subset == 'training':
                self.dataframe = train_dataset
            elif self.subset == 'validation':
                self.dataframe = valid_dataset
        
        self.n = len(self.dataframe)

    def pd_from_path(self, directory: str = './') -> pd.DataFrame:
        self.class_to_id = {}
        data = []
        for path, _, files in os.walk(directory):
            for file in files:
                if file.endswith(('.jpg', '.png', '.jpeg', '.bmp', '.gif')):
                    class_name = os.path.basename(path)
                    if class_name not in self.class_to_id:
                        self.class_to_id[class_name] = len(self.class_to_id)
                    data.append((f'{path}/{file}', self.class_to_id[class_name]))

        return pd.DataFrame(data, columns=['imagepath', 'label'])\
    
    def tf_load_image(self, image_path: str, crop_size: tuple[int, int] = (256, 256)):
        image = tf.io.read_file(image_path)
        image = tf.io.decode_image(image, expand_animations=False, channels=3)
        image = tf.image.convert_image_dtype(image, dtype=tf.float32)
        image = tf.image.resize(image, size=[*crop_size])
        return self.augment(image) if self.augmentation else image
    
    def augment(self, image):
        if tf.random.uniform((), maxval=1) > 0.5:
            return tf.image.random_flip_left_right(image)
        return tf.image.random_flip_up_down(image)
    
    def __len__(self):
        return self.n // self.batch_size

    def __getitem__(self, index):
        items_from_dataset = self.dataframe[index * self.batch_size:(index + 1) * self.batch_size]
    
        return tf.stack(list(map(self.tf_load_image, items_from_dataset.imagepath))), tf.stack(items_from_dataset.label)

    def on_epoch_end(self):
        print("Hooray, IT'S ALIVE")

In [26]:
dataset3 = DatasetFromDirectory(path)

In [6]:
inputs3 = tf.keras.layers.Input((256, 256, 3), name='input')
x3 = tf.keras.layers.Conv2D(32, 3, padding='same')(inputs3)
x3 = tf.keras.layers.PReLU()(x3)
x3 = tf.keras.layers.Flatten()(x3)
x3 = tf.keras.layers.Dense(64)(x3)

In [7]:
model3 = tf.keras.Model(inputs=inputs3, outputs=x3, name='ciaonima-1')
model3.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics='accuracy', optimizer='adam')

In [23]:
model3.fit(dataset3, epochs=1)

2808/2808 [==============================] - 487s 172ms/step - loss: nan - accuracy: 0.0019
Hooray, IT'S ALIVE


In [82]:
class DatasetFromCSV(tf.keras.utils.Sequence):
    
    def __init__(self,
                 path: str = './',
                 batch_size: int = 32,
                 image_size: tuple[int, int] = (256, 256),
                 shuffle: bool = True,
                 split: str = 'train',
                 augmentation: bool = True
                 ):
        self.directory = path
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle
        self.split = split
        self.augmentation = augmentation

        self.dataframe = self.pd_from_csv(self.directory)
        
        if self.shuffle:
            self.dataframe = self.dataframe.sample(frac=1)

        if self.split:
            self.dataframe = self.dataframe[self.dataframe['data set'] == self.split]

        self.n = len(self.dataframe)

    def pd_from_csv(self, path):
        data = pd.read_csv(path)
        data.filepaths = [os.path.join(os.path.dirname(path), file).replace('\\', '/') for file in data.filepaths.tolist()]
        classes = pd.unique(data.labels)
        class_to_id = {name: i for i, name in enumerate(classes)}
        data.insert(3, 'labels_id', [class_to_id[name] for name in data.labels], True)
        return data
    
    def tf_load_image(self, image_path: str, crop_size: tuple[int, int] = (256, 256)):
        image = tf.io.read_file(image_path)
        image = tf.io.decode_image(image, expand_animations=False, channels=3)
        image = tf.image.convert_image_dtype(image, dtype=tf.float32)
        image = tf.image.resize(image, size=[*crop_size])
        return self.augment(image) if self.augmentation else image
    
    def augment(self, image):
        if tf.random.uniform((), maxval=1) > 0.5:
            return tf.image.random_flip_left_right(image)
        return tf.image.random_flip_up_down(image)
    
    def __len__(self):
        return self.n // self.batch_size
    
    def __getitem__(self, index):
        items_from_dataset = self.dataframe[index * self.batch_size:(index + 1) * self.batch_size]
    
        return tf.stack(list(map(self.tf_load_image, items_from_dataset.filepaths))), tf.stack(items_from_dataset.labels_id)

    def on_epoch_end(self):
        print("Hooray, IT'S ALIVE!?")

In [83]:
dataset4 = DatasetFromCSV('./archive/new_birds.csv')

In [6]:
inputs4 = tf.keras.layers.Input((256, 256, 3), name='input')
x4 = tf.keras.layers.Conv2D(32, 3, padding='same')(inputs4)
x4 = tf.keras.layers.PReLU()(x4)
x4 = tf.keras.layers.Flatten()(x4)
x4 = tf.keras.layers.Dense(64)(x4)

In [7]:
model4 = tf.keras.Model(inputs=inputs4, outputs=x4, name='ciaonima-1')
model4.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics='accuracy', optimizer='adam')

In [84]:
model4.fit(dataset4, epochs=1)

2644/2644 [==============================] - 439s 166ms/step - loss: nan - accuracy: 0.0019
Hooray, IT'S ALIVE!?
